In [1]:
#importing required libraries
import glob
from imageai.Detection import VideoObjectDetection
import natsort
import numpy as np
import os
import sys
import numpy as np
import cv2
import glob
from imageai.Detection import VideoObjectDetection
import os
import sys
from random import randint
from math import ceil, nan, sqrt
import natsort
import pandas as pd
import random
from sklearn.svm import SVC 
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import GridSearchCV
import sys
import cv2
from random import randint
import re

Using TensorFlow backend.


In [ ]:
def get_numbers_from_filename(filename):
    'utility function to get first number from a file name'
    return re.search(r'\d+', filename).group(0)

trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']

def createTrackerByName(trackerType):
    
  # Create a tracker based on tracker name
    if trackerType == trackerTypes[0]:
        tracker = cv2.TrackerBoosting_create()
  
    elif trackerType == trackerTypes[1]: 
        tracker = cv2.TrackerMIL_create()
   
    elif trackerType == trackerTypes[2]:
        tracker = cv2.TrackerKCF_create()
  
    elif trackerType == trackerTypes[3]:
        tracker = cv2.TrackerTLD_create()
   
    elif trackerType == trackerTypes[4]:
        tracker = cv2.TrackerMedianFlow_create()
    
    elif trackerType == trackerTypes[5]:
        tracker = cv2.TrackerGOTURN_create()
            
    elif trackerType == trackerTypes[6]:
        tracker = cv2.TrackerMOSSE_create()
        
    elif trackerType == trackerTypes[7]:
        tracker = cv2.TrackerCSRT_create()
        
    else:
        tracker = None
        print('Incorrect tracker name')
        print('Available trackers are:')
        for t in trackerTypes:
            print(t)
     
    return tracker

In [ ]:
def get_speeds_from_frame(video, boxes_list, frame_no, no_previous_frames, trackerType = "KCF"):

    '''
    Detects speed of each vehicle in the frame and returns and an containing these speeds

    Parameters:
    video(string) : path of the video
    frame_no : frame no of the video for which you require directions
    boxes_list(list) : list containing bounding boxes for all detected vehicles in the particular frame
    no_previous_frames(int) : number of previous frames to be considered while calculating the speed
    trackerType(string) : the openCV tracker type to be used

    '''
    frame_buffer = []
    #cv2.namedWindow(video, cv2.WINDOW_NORMAL)
    #cv2.resizeWindow(video, height=650, width=1156)
    cap = cv2.VideoCapture(video)

    for i in range(frame_no - no_previous_frames, frame_no + 1):

        cap.set(1,i)
        ret, frame = cap.read()
        frame_buffer.append(frame)

    multiTracker_back = cv2.MultiTracker_create()
    #multiTracker = cv2.MultiTracker_create()
    n = len(boxes_list)
    speeds = [0]*n
    x1 = [0]*n  #x coordinates of vehicles in frame_no
    y1 = [0]*n #y coordinates of vehicles in frame_no
    x2 = [0]*n  #x coordinates of vehicles in frame_no - no_previous_frames
    y2 = [0]*n #y coordinates of vehicles in frame_no - no_previous_frames

    frame = frame_buffer.pop()
    for i in range(n):
        box = boxes_list[i]
        x1[i] = (box[0]+2*box[2])/2
        y1[i] = (box[1]+2*box[3])/2
        multiTracker_back.add(createTrackerByName(trackerType), frame, box)
        #all the vehicles from boxes_list added in the traker instance
        #applying vehicle tracking in backward direction
    for j in range(no_previous_frames):
        frame = frame_buffer.pop()
        (success_back, boxes_back) = multiTracker_back.update(frame)

    for i in range(n):
        b = boxes_back[i]
        x2[i] = (b[0]+2*b[2])/2
        y2[i] = (b[1]+2*b[3])/2
        x_d = x2[i] - x1[i]
        y_d = y2[i] - y1[i]
        dist = (x_d**2 + y_d**2)**0.5
        speeds[i] = dist

    return np.array(speeds)


In [ ]:
def get_speeds_from_video(video, detection_array, target_folder, no_previous_frames = 6):

    '''
    Detects speed of each vehicle in each frame of the video and returns and an containing these speeds

    Parameters:
    video(string) : path of the video
    detection_array(string) : path of the detection_array
    target_folder(string) : path of the folder where you want to store the array of speeds
    no_previous_frames(int) : number of previous frames to be considered while calculating the speed

    '''

    boxes_array = np.load(detection_array, allow_pickle=True)
    no_frames = boxes_array.shape[0]
    print("no_frames: ", no_frames)
    findex = get_numbers_from_filename(video)
    S = []
    for i in range(no_previous_frames):
        S.append([])

    for frame_no in range(no_previous_frames, no_frames):
        print("processing frame_no: ", frame_no)

        temp = boxes_array[frame_no]
        boxes_list = [(b[0],b[1],b[2]-b[0],b[3]-b[1]) for b in temp]

        speeds = get_speeds_from_frame(video, boxes_list, frame_no, no_previous_frames)
        S.append(speeds)
            
    S = np.array(S)
    print('saving speeds for video' + video)
    np.save(target_folder+'speeds'+str(findex),S)


In [ ]:
videos_folder = "C:/RoadCrossingAssistant/Data/Videos/"
videos = glob.glob(videos_folder+'video*.MOV')
videos = natsort.natsorted(videos)
arrays_folder = "C:/RoadCrossingAssistant/Data/Arrays_RetinaNet/"
arrays = glob.glob(arrays_folder+'array*.npy')
arrays = natsort.natsorted(arrays)
target_folder = "C:/RoadCrossingAssistant/Data/Speeds_RetinaNet/"

for (vname,aname) in zip(videos,arrays):
    print("starting for {} \n" .format(vname))
    get_speeds_from_video(vname, aname, target_folder)
    print("----------------------------")


In [2]:
def visualize_speeds_video(video, detection_array, speeds_array):

    '''
    Visualize calculated speeds for the video (press a to start the display of visualization in the opencv window)

    Parameters:
    video(string) : path of the video
    detection_array(string) : path of the detection_array
    speeds_array(string) : path of the speeed array

    '''
    cap = cv2.VideoCapture(video)
    cv2.namedWindow(video, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(video, height=650, width=1156)
    no_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    detection_array = np.load(detection_array, allow_pickle=True)
    speeds_array = np.load(speeds_array,allow_pickle=True)

    frame_count = -1

    while(cap.isOpened()):

        success,frame = cap.read()
        key = cv2.waitKey(100)
        #print(i)

        if success == True:

            if(frame_count == -1):
                while True:

                    key_init = cv2.waitKey(25)
                    cv2.imshow(video, frame)

                    if key_init == ord('a'):
                        break

            frame_count = frame_count + 1

            speeds_frame = speeds_array[frame_count]
            temp = detection_array[frame_count]
            boxes_frame = [(b[0],b[1],b[2]-b[0],b[3]-b[1]) for b in temp]
            no_boxes = len(boxes_frame)

            if(frame_count < 6):

                for i in range(no_boxes):
                    box = boxes_frame[i]
                    (x,y,w,h) = [int(v) for v in box]
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)

                cv2.imshow(video, frame)
                cv2.waitKey(100)        

            else:
                if(len(speeds_frame) != len(boxes_frame)):
                    raise Exception("mismatch in length of speeds_frame and boxes_frame")
                
                for i in range(no_boxes):
                    box = boxes_frame[i]
                    (x,y,w,h) = [int(v) for v in box]
                    s = round(speeds_frame[i], 2)
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
                    cv2.putText(frame, str(s), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)
                cv2.imshow(video, frame)

                if key == ord('p'): # pause a video

                    while True:
                        
                        key2 = cv2.waitKey(33)
                        cv2.imshow(video, frame)
                        
                        if key2 == ord('p'): # resume video after pausing
                            break

        else:
            break

    
    cap.release()
    cv2.destroyAllWindows()

In [3]:
vname = 'C:/RoadCrossingAssistant/Data/Videos/video64.MOV'
aname = 'C:/RoadCrossingAssistant/Data/Arrays_RetinaNet/array64.npy'
sname = 'C:/RoadCrossingAssistant/Data/Speeds_RetinaNet/speeds64.npy'
visualize_speeds_video(vname, aname, sname)

In [4]:
#extra........
def visualize_video(video, detection_array):

    cap = cv2.VideoCapture(video)
    cv2.namedWindow(video, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(video, height=650, width=1156)
    no_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    detection_array = np.load(detection_array, allow_pickle=True)

    frame_count = -1

    while(cap.isOpened()):

        success,frame = cap.read()
        key = cv2.waitKey(34)
        #print(i)

        if success == True:

            if(frame_count == -1):
                while True:

                    key_init = cv2.waitKey(25)
                    cv2.imshow(video, frame)

                    if key_init == ord('a'):
                        break

            if key == ord('p'): # pause a video

                while True:
                    
                    key2 = cv2.waitKey(33)
                    cv2.imshow(video, frame)
                    
                    if key2 == ord('p'): # resume video after pausing
                        break

            frame_count = frame_count + 1

            temp = detection_array[frame_count]
            boxes_frame = [(b[0],b[1],b[2]-b[0],b[3]-b[1]) for b in temp]
            print("frame: ", frame_count)
            no_boxes = len(boxes_frame)
                
            
            for i in range(no_boxes):
                box = boxes_frame[i]
                (x,y,w,h) = [int(v) for v in box]
                cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
    
            cv2.imshow(video, frame)
            cv2.waitKey(100)            

        else:
            break

    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
visualize_video(vname, fname)

In [ ]:
detection_array = np.load(fname, allow_pickle=True)
print(detection_array.shape)
print(detection_array[:15])